In [1]:
import torch
import numpy as np
import pandas as pd 

from torch import nn
from torch.utils.data import \
    Dataset as DS, \
    DataLoader as DL
from pandas import read_csv
from sklearn.model_selection import train_test_split
from ipdb import set_trace

import utils


### load data

In [2]:
train_data = read_csv(f"{utils._data_pth_}/processed/train_joined.csv", index_col=0)
y, X = train_data['isFraud'], train_data.drop(columns=['isFraud'])
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=utils._random_seed_)

In [3]:
print(len(X_train[X_train.card1>0]), len(X_train[X_train.card1<0]))
print(len(Y_test[Y_test==1]), len(Y_test[Y_test==0]))

194324 201337
6778 188101


### model

In [7]:
class AlexNet(torch.nn.Module):
    def __init__(self, num_classes: int = 2, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv1d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Conv1d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Conv1d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv1d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool1d(6)
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        x = torch.flatten(x, 0)
        return x


### dataset

In [ ]:
class Fraud_Dataset(DS):
    def __init__(self, X: pd.DataFrame, y: pd.DataFrame):
        self.X = X.values.astype(np.float32)
        self.Y = y.values.astype(np.float32)
    
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


### training configuration

In [9]:
dropout = 0.5
lr = 0.0002
batch_size = 100
num_classes = 1
epochs = 10

### training

In [10]:
train_ds = Fraud_Dataset(X_train, Y_train)
train_dl = DL(train_ds, batch_size=batch_size, shuffle=True)

AN = AlexNet(num_classes=num_classes, dropout=dropout)
AN_opt = torch.optim.Adam(AN.parameters(), lr=lr)
if torch.cuda.is_available():
    AN.cuda()
    
sigmoid = nn.Sigmoid() # final activation
bce = nn.BCELoss() # binary cross-entropy 

    
for e in range(epochs):
    
    # loss data
    loss_df = pd.DataFrame({"loss": []})
    
    for i, (X, y) in enumerate(train_dl):

        if torch.cuda.is_available():
            X = X.cuda()
            y = y.cuda()
        X = X.unsqueeze(1) # add a channel dimension
        
        AN_opt.zero_grad()
        
        y_probs = AN(X)
        loss = bce(sigmoid(y_probs), y)
        loss.backward() 
        loss_df = loss_df.append({"loss": loss.item()}, ignore_index=True)
        
        AN_opt.step()
        
    torch.save(AN.state_dict(), f"{utils._data_pth_}/models/AN_epochs_{e}.pth")
    loss_df.to_csv(f"{utils._data_pth_}/models/AN_epochs_{e}_loss.csv")


In [12]:
AN = AlexNet(num_classes=num_classes, dropout=dropout)
model = torch.load(f'{utils._data_pth_}/models/AN_epochs_0.pth', "cuda" if torch.cuda.is_available() else "cpu") 
AN(torch.Tensor(X_test.values).unsqueeze(1))
y_probs = AN(X_test)



--Return--
None
> /tmp/ipykernel_25702/3960282452.py(4)<module>()
      3 AN.load_state_dict(model)
----> 4 set_trace()
      5 y_probs = AN(X_test)

ipdb> type(X_test)
<class 'pandas.core.frame.DataFrame'>
ipdb> AN(X_test.values)
*** TypeError: conv1d() received an invalid combination of arguments - got (numpy.ndarray, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!numpy.ndarray!, !Parameter!, !Parameter!, !tuple!, !tuple!, !tuple!, int)
ipdb> X_test.shape
(194879, 432)
ipdb> torch.Tensor(X_test.values).unsquee

BdbQuit: 

In [ ]:
torch.cuda.is_available()
